## Movie Recommendation
### 프로젝트 - Movielens 영화 추천 실습
***
> **1) 데이터 준비와 전처리
> Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.**  
> 
> **2) 분석해 봅시다.  
> -ratings에 있는 유니크한 영화 개수  
> -rating에 있는 유니크한 사용자 수  
> -가장 인기 있는 영화 30개(인기순)**  
> 
> **3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.**  
> **4) CSR matrix를 직접 만들어 봅시다.**  
> **5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.**  
> **6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.**  
> **7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.**  
> **8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.**
***
**평가문항**  
> ***1. CSR matrix가 정상적으로 만들어졌다.  
> -사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.***  
>  
> ***2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.   
> -사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.***  
>  
> ***3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
> -MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.***  
***
### **1) 데이터 준비와 전처리**

In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings에서 timestamp 컬럼 삭제
ratings.drop(['timestamp'], axis=1, inplace=True)

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) 분석해 봅시다. 
**-ratings에 있는 유니크한 영화 개수  
-rating에 있는 유니크한 사용자 수  
-가장 인기 있는 영화 30개(인기순)**  

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자수
ratings['user_id'].nunique()

6039

In [8]:
# 합쳐주기
ratings_title= pd.merge(ratings, movies[['title','movie_id']], on='movie_id', how='left')
ratings_title

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)


In [9]:
# ratings에 가장 인기많은 영화
movie_count = ratings_title.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3) 선호하는 영화 5개 추가하기

In [10]:
# 내가 좋아하는 영화 5개 고르기
my_favorite = ['Home Alone (1990)' , 'Snow White and the Seven Dwarfs (1937)' ,'Aladdin (1992)' ,'Pinocchio (1940)' ,'Beauty and the Beast (1991)']

# 'ethan'이라는 user_id가 위영화의 평가점수를 4점씩 주었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['ethan']*5, 'title': my_favorite, 'count':[4]*5})

if not ratings_title.isin({'user_id':['ethan']})['user_id'].any():  # user_id에 'ethan'이라는 데이터가 없다면
    ratings_title = ratings_title.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings_title.tail(10) 

,user_id,movie_id,count,title
836473,6040,1090.0,3,Platoon (1986)
836474,6040,1094.0,5,"Crying Game, The (1992)"
836475,6040,562.0,5,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,Sophie's Choice (1982)
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982)
0,ethan,NaN,4,Home Alone (1990)
1,ethan,NaN,4,Snow White and the Seven Dwarfs (1937)
2,ethan,NaN,4,Aladdin (1992)
3,ethan,NaN,4,Pinocchio (1940)
4,ethan,NaN,4,Beauty and the Beast (1991)


In [11]:
# index번호를 다시 매겨주기
ratings_title.reset_index(drop=True, inplace=True)
ratings_title.tail(10)

,user_id,movie_id,count,title
836473,6040,1090.0,3,Platoon (1986)
836474,6040,1094.0,5,"Crying Game, The (1992)"
836475,6040,562.0,5,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,Sophie's Choice (1982)
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982)
836478,ethan,NaN,4,Home Alone (1990)
836479,ethan,NaN,4,Snow White and the Seven Dwarfs (1937)
836480,ethan,NaN,4,Aladdin (1992)
836481,ethan,NaN,4,Pinocchio (1940)
836482,ethan,NaN,4,Beauty and the Beast (1991)


**movie_id의 NaN값을 영화의 movie_id로 대체하기 위해 title merge 실행합니다.**

In [12]:
ratings_title_num= pd.merge(ratings_title, movies[['title','movie_id']], on='title', how='left')
ratings_title_num.tail(10)

,user_id,movie_id_x,count,title,movie_id_y
836473,6040,1090.0,3,Platoon (1986),1090
836474,6040,1094.0,5,"Crying Game, The (1992)",1094
836475,6040,562.0,5,Welcome to the Dollhouse (1995),562
836476,6040,1096.0,4,Sophie's Choice (1982),1096
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,NaN,4,Home Alone (1990),586
836479,ethan,NaN,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,NaN,4,Aladdin (1992),588
836481,ethan,NaN,4,Pinocchio (1940),596
836482,ethan,NaN,4,Beauty and the Beast (1991),595


In [13]:
# ratings_title_num에서 movie_id_x 컬럼 삭제
ratings_title_num.drop(['movie_id_x'], axis=1, inplace=True)
# movie_id_y 컬럼의 이름을 movie_id로 바꿉니다.
ratings_title_num.rename(columns={'movie_id_y':'movie_id'}, inplace=True)

In [14]:
ratings_title_num.tail(10)

,user_id,count,title,movie_id
836473,6040,3,Platoon (1986),1090
836474,6040,5,"Crying Game, The (1992)",1094
836475,6040,5,Welcome to the Dollhouse (1995),562
836476,6040,4,Sophie's Choice (1982),1096
836477,6040,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,4,Home Alone (1990),586
836479,ethan,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,4,Aladdin (1992),588
836481,ethan,4,Pinocchio (1940),596
836482,ethan,4,Beauty and the Beast (1991),595


In [15]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings_title_num['user_id'].unique()
movie_unique = ratings_title_num['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
# 고유한 유저id와 영화타이틀의 객체:인덱스 순으로 하는 딕셔너리를 각각 생성합니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [40]:
print(movie_to_idx)

{"One Flew Over the Cuckoo's Nest (1975)": 0, 'James and the Giant Peach (1996)': 1, 'My Fair Lady (1964)': 2, 'Erin Brockovich (2000)': 3, "Bug's Life, A (1998)": 4, 'Princess Bride, The (1987)': 5, 'Ben-Hur (1959)': 6, 'Christmas Story, A (1983)': 7, 'Snow White and the Seven Dwarfs (1937)': 8, 'Wizard of Oz, The (1939)': 9, 'Beauty and the Beast (1991)': 10, 'Gigi (1958)': 11, 'Miracle on 34th Street (1947)': 12, "Ferris Bueller's Day Off (1986)": 13, 'Sound of Music, The (1965)': 14, 'Airplane! (1980)': 15, 'Tarzan (1999)': 16, 'Bambi (1942)': 17, 'Awakenings (1990)': 18, 'Big (1988)': 19, 'Pleasantville (1998)': 20, 'Wallace & Gromit: The Best of Aardman Animation (1996)': 21, 'Back to the Future (1985)': 22, "Schindler's List (1993)": 23, 'Meet Joe Black (1998)': 24, 'Pocahontas (1995)': 25, 'E.T. the Extra-Terrestrial (1982)': 26, 'Titanic (1997)': 27, 'Ponette (1996)': 28, 'Close Shave, A (1995)': 29, 'Antz (1998)': 30, 'Girl, Interrupted (1999)': 31, 'Hercules (1997)': 32, 'Al

In [16]:
print(user_to_idx['ethan'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['Aladdin (1992)'])

6039
33


In [17]:
ratings_title_num.tail(10)

,user_id,count,title,movie_id
836473,6040,3,Platoon (1986),1090
836474,6040,5,"Crying Game, The (1992)",1094
836475,6040,5,Welcome to the Dollhouse (1995),562
836476,6040,4,Sophie's Choice (1982),1096
836477,6040,4,E.T. the Extra-Terrestrial (1982),1097
836478,ethan,4,Home Alone (1990),586
836479,ethan,4,Snow White and the Seven Dwarfs (1937),594
836480,ethan,4,Aladdin (1992),588
836481,ethan,4,Pinocchio (1940),596
836482,ethan,4,Beauty and the Beast (1991),595


In [18]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅니다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings_title_num['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_title_num):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings_title_num['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings_title_num['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings_title_num):
    print('amovie column indexing OK!!')
    ratings_title_num['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings_title_num

user_id column indexing OK!!
amovie column indexing OK!!


,user_id,count,title,movie_id
0,0,5,0,1193
1,0,3,1,661
2,0,3,2,914
3,0,4,3,3408
4,0,5,4,2355
...,...,...,...,...
836478,6039,4,507,586
836479,6039,4,8,594
836480,6039,4,33,588
836481,6039,4,528,596


In [20]:
ratings_title_num['count']

0         5
1         3
2         3
3         4
4         5
         ..
836478    4
836479    4
836480    4
836481    4
836482    4
Name: count, Length: 836483, dtype: int64

In [21]:
print(ratings_title_num['count'].values)

[5 3 3 ... 4 4 4]


### 4) CSR matrix 만들기

In [19]:
from scipy.sparse import csr_matrix

num_user = ratings_title_num['user_id'].nunique()
num_movie = ratings_title_num['title'].nunique()

csr_data = csr_matrix((ratings_title_num['count'].values, (ratings_title_num.user_id, ratings_title_num.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장사항입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 5) als_model = AlternatingLeastSquares 모델
**-모델 구성**

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=256, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

**-모델훈련**  

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 모델이 예측한 나의 선호도 (feat. 내가 고른 영화)  

In [25]:
ethan, aladdin = user_to_idx['ethan'], movie_to_idx['Aladdin (1992)']
ethan_vector, aladdin_vector = als_model.user_factors[ethan], als_model.item_factors[aladdin]

In [26]:
ethan_vector

array([ 0.16878335, -0.09989601, -0.00656716, -0.364013  , -0.35038227,
       -0.03823595,  0.46970242,  0.07276542,  0.26608145, -0.2567468 ,
        0.17191592,  0.24658924,  0.7649133 , -0.18750603,  0.25323907,
        0.47599253,  0.33700818,  0.25317615,  0.2822545 ,  0.6424485 ,
        0.47869882, -0.6672743 ,  0.05324005, -0.08381694,  0.23700048,
       -0.12106617,  0.1360133 , -0.46829435,  0.12089088, -0.33267418,
        0.16769028, -0.4337841 , -0.10234536, -0.11955222,  0.19745553,
       -0.41247955,  0.05009748,  0.00661483,  0.03483262, -0.17726259,
       -0.38755763,  0.12813981,  0.06570356, -0.21036449, -0.06907293,
       -0.436628  ,  0.5636173 , -0.2241512 ,  0.25216573, -0.17214409,
       -0.22522661, -0.453097  ,  0.1258827 ,  0.48927772, -0.14931783,
       -0.02632755, -0.20953436,  0.42855868,  0.40300122,  0.3456833 ,
        0.2789519 ,  0.05802483, -0.12599835,  0.1452229 , -0.33800736,
        0.09330316,  0.5184351 ,  0.9600234 , -0.01674435,  0.55

In [27]:
aladdin_vector

array([ 8.48150533e-03,  1.49931002e-03,  7.72980461e-03, -4.91042063e-03,
        2.44047754e-02,  1.39107425e-02,  2.32929606e-02,  1.72059145e-02,
        1.40082380e-02,  2.42114393e-03,  2.37087458e-02,  1.01528792e-02,
        2.68391799e-02, -5.12562227e-03,  1.25188809e-02,  1.54572036e-02,
       -8.44397862e-03,  1.00419531e-02, -6.98454864e-03,  3.91673204e-03,
        3.16716265e-03, -1.15822162e-02,  8.90904665e-03,  1.09167937e-02,
        2.23749280e-02,  2.21968643e-04,  1.95232425e-02,  1.31720416e-02,
        1.60540733e-02, -1.40996389e-02,  2.00538561e-02, -1.47780851e-02,
        1.79427909e-03, -1.02542322e-02,  1.52554773e-02, -5.06960554e-03,
        1.21418927e-02,  8.90123993e-05,  4.07998962e-03, -3.13437311e-03,
        2.57951627e-03,  1.26045598e-02, -9.89388302e-03, -3.62097868e-03,
        2.39234394e-03,  1.66985579e-02,  2.00691782e-02, -7.11701345e-03,
        5.49980253e-03,  1.59708806e-03,  9.48011118e-04, -1.07032126e-02,
        9.13183670e-03,  

In [28]:
np.dot(ethan_vector, aladdin_vector)

0.7066561

**기존에 선호한다고 골랐던 영화와의 내적값이 0.709687로 나름 의미있게 내적값이 나왔다고 판단됩니다.**

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기.

In [29]:
# 객체 : 인덱스번호 순으로 되어있던것을 인덱스번호 : 객체순으로 변경
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [32]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

**위에서 선언했던 my_favorite리스트를 활용하여 내가 좋아하는 영화와 비슷한 영화를 추천받겠습니다.  
my_favorite = ['Home Alone (1990)' , 'Snow White and the Seven Dwarfs (1937)' ,'Aladdin (1992)' ,'Pinocchio (1940)' ,'Beauty and the Beast (1991)']**

In [41]:
get_similar_movie(my_favorite[0])

['Home Alone (1990)',
 'Home Alone 2: Lost in New York (1992)',
 'Mighty Ducks, The (1992)',
 'D2: The Mighty Ducks (1994)',
 'Home Alone 3 (1997)',
 'Flintstones, The (1994)',
 '101 Dalmatians (1996)',
 'Big Green, The (1995)',
 'D3: The Mighty Ducks (1996)',
 'Angels in the Outfield (1994)']

In [42]:
get_similar_movie(my_favorite[1])

['Snow White and the Seven Dwarfs (1937)',
 'Dumbo (1941)',
 'Cinderella (1950)',
 'Pinocchio (1940)',
 'Sleeping Beauty (1959)',
 'Peter Pan (1953)',
 'Bambi (1942)',
 'Alice in Wonderland (1951)',
 'Fantasia (1940)',
 'Jungle Book, The (1967)']

In [43]:
get_similar_movie(my_favorite[2])

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Little Mermaid, The (1989)',
 'Anastasia (1997)',
 'Mulan (1998)',
 'Hunchback of Notre Dame, The (1996)',
 'Mass Appeal (1984)',
 'Man of the Century (1999)',
 "Airport '77 (1977)"]

In [44]:
get_similar_movie(my_favorite[3])

['Pinocchio (1940)',
 'Bambi (1942)',
 'Dumbo (1941)',
 'Snow White and the Seven Dwarfs (1937)',
 'Cinderella (1950)',
 'Sleeping Beauty (1959)',
 'Sword in the Stone, The (1963)',
 'Robin Hood (1973)',
 'Peter Pan (1953)',
 '101 Dalmatians (1961)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아보기.

In [45]:
# user를 기반으로 내적값을 구해 높은 순서대로 20개를 리스트화 합니다.
user = user_to_idx['ethan']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(330, 0.5303469),
 (17, 0.41167593),
 (37, 0.3631115),
 (46, 0.3452391),
 (191, 0.33034673),
 (548, 0.31833413),
 (572, 0.28238276),
 (551, 0.2614317),
 (536, 0.24259107),
 (45, 0.24067038),
 (40, 0.2368128),
 (950, 0.22810069),
 (619, 0.22366321),
 (34, 0.22261836),
 (520, 0.21154492),
 (547, 0.20672047),
 (559, 0.1587121),
 (284, 0.1548994),
 (16, 0.14901704),
 (936, 0.13937679)]

In [46]:
# movie_id로 되어있던 값을 movie_title로 바꿔줍니다.
[idx_to_movie[i[0]] for i in movie_recommended]

['Lion King, The (1994)',
 'Bambi (1942)',
 'Cinderella (1950)',
 'Dumbo (1941)',
 'Little Mermaid, The (1989)',
 'Fantasia (1940)',
 'Sleeping Beauty (1959)',
 'Peter Pan (1953)',
 'Jungle Book, The (1967)',
 'Mary Poppins (1964)',
 'Toy Story (1995)',
 '101 Dalmatians (1961)',
 'Alice in Wonderland (1951)',
 'Mulan (1998)',
 "Charlotte's Web (1973)",
 'Lady and the Tramp (1955)',
 'Robin Hood (1973)',
 'Nightmare Before Christmas, The (1993)',
 'Tarzan (1999)',
 'Secret of NIMH, The (1982)']

In [48]:
# 내가 기록을 남긴 데이터 중에 이 추천에 기여한 정도를 확인합니다
Lion_King = movie_to_idx['Lion King, The (1994)']
explain = als_model.explain(user, csr_data, itemid=Lion_King)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Beauty and the Beast (1991)', 0.23689308431068265),
 ('Aladdin (1992)', 0.20198531703567701),
 ('Snow White and the Seven Dwarfs (1937)', 0.06911801841374189),
 ('Home Alone (1990)', 0.021438585161107478),
 ('Pinocchio (1940)', -0.0060303075237104434)]

### 회고록
**벌써 1월이 지나고 2월이 되었다. 정말 정신없이 1월이 지나간 것 같다.  
2월에 제출하는 두번째 프로젝트로 만든 영화추천시스템은 생각보다 재밌었다.  
물론 재미만 있었던 것은 아니었다. 생각보다 판다스 데이터 프레임을 다루는게 어려웠다.  
아직 숙련도가 떨어져서 머릿속에서 이렇게 정리해야지 하는부분이 없는 것 같다.  
덕분에 창 한쪽에는 언제나 구글이 켜져있었다.  
그래도 하나하나 구글링해가면서 이해하고 적용해보고 그게 생각대로 되었을 때는 짜릿했다.  
앞으로 진행하게 될 프로젝트들도 이렇게라도 해결할 수 있기를 바래본다.**  